<a href="https://colab.research.google.com/github/collinjennings/detectiveLLMs/blob/main/chatInterfaceTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mistral v.03 Instructional Model Test
9.9.24

Here I tested the Mistral v.3 Instruct model on one Sherlock Holmes short story. I ran two tests.
1. **Full story** - I gave the model the full story and asked it to identify the perpetrator and a key clue for identifying him.
2. **Cropped story** - I cropped the story at the start of the 'reveal' portion. I then asked the model to predict the perpetrator and identify a piece of evidence it used to make that prediction.

**Result**: With the full story, the model correctly identified the perpetrator and one important clue (although not the most important one). With the cropped story, the model identified the a key suspect and clue, but it was one that was already addressed by Holmes in that portion of the story, before he arrives at the actual criminal, who also happened to be the victim of the murder.

From this initial test, it appears that the model is still primarily summarizing what it has rather than making an actual prediction regarding what it doesn't know. A human reader would most likely know that the perpetrator couldn't be the person identified by the model because Holmes had already addressed his level of culpability. It wouldn't make sense in terms of the plot mechanics of the story.

*Note: I ran the model using A100 GPU through Google Colab.*

In [2]:
import glob
from collections import defaultdict
import re
import os
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
import math
import os
import torch
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2,3'

#### Mounting Google drive to load the corpus.

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/Colab Notebooks'

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks


In [17]:
from huggingface_hub import login
login(token="")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


#### Loading the tokenizer and model.

In [19]:
max_token_length = 12000

In [20]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3", model_max_length = max_token_length, cache_dir= None)
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3", pad_token_id = tokenizer.eos_token_id,cache_dir= None, device_map="auto")

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [4]:
### Loading the corpus.
import glob
files = glob.glob("./data/texts/*.txt")
texts = defaultdict()
for afile in files:
      texts[re.sub('_','', afile.split('/')[3].split('.')[0])] = open(afile, encoding = 'utf-8').read()

In [5]:
### Identifying the line where the solution reveal starts.
for idx, line in enumerate(texts['MSH01'].split('.')):
  if 'Sherlock Holmes laughed' in line:
    print(idx, line)


496 ”
Sherlock Holmes laughed


In [15]:
### Crop the story at the reveal border sentence.
text = texts['MSH01'].split('.')[:496]
len(text)
print(text[495])
text = '.'.join(text)

 Holmes,” said he, “but I must regard what you have just said as either a very bad joke or an insult


#### Running the story and prompt through the model.

In [21]:
### Run the story throught the model, and feed it the instruction to predict the
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
B_INST, E_INST = "[INST] ", " [/INST]"
prompt = f"{B_INST} Read the excerpt from the following detective story, and predict who committed the crime. Then identify one key piece of evidence that supports your prediction:\n\n[TEXT_START]\n\n{text}\n\n[TEXT_END]\n\n{E_INST}"
inputs = tokenizer( prompt,return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=max_token_length, use_cache=False, do_sample=True,temperature=0.2, top_p=0.95)
prompt_length = inputs['input_ids'].shape[1] #del ['input_ids]
summary = tokenizer.decode(outputs[0][prompt_length:-1])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


#### Model Prediction

**Prompt**: Read the excerpt from the following detective story, and predict who committed the crime. Then identify one key piece of evidence that supports your prediction.

**Prediction**:

In [22]:
print(summary)

Based on the information provided in the excerpt, it appears that the murderer of John Straker and the thief of Silver Blaze is Silas Brown, the trainer of Desborough, Lord Backwater's horse. One key piece of evidence that supports this prediction is the fact that Holmes discovers the tracks of a horse and a man leading away from the scene of the crime towards Mapleton, where Silas Brown's stables are located. Additionally, Holmes confronts Silas Brown and is able to extract a confession from him, revealing that he had led Silver Blaze away to hide it until after the race, and had also killed John Straker in order to prevent him from interfering.


#### Model Identification of Murderer and key clue from full story

**Prompt**: Read the following detective story and identify one key clue used to solve the mystery described in the story.

**Answer**: One key clue used to solve the mystery described in the story is the curried mutton that was served to the stable-boy, Hunter, on the night of the crime. The mutton was found to contain an appreciable quantity of powdered opium, which was used to drug Hunter and make him unconscious, allowing the thief to take the horse without being detected. This clue helps Holmes to eliminate Fitzroy Simpson as a suspect, as it is unlikely that Simpson would have had access to the curried mutton without the maid seeing him. Instead, the focus shifts to John Straker and his wife, who were the only two people who could have chosen curried mutton for supper that night and had access to the dish without the maid seeing them. This clue ultimately leads Holmes to discover that Straker was leading a double life, keeping a second establishment, and that he was the one who took out the horse to a hollow where he intended to injure it.